# Introduction to Listening Machines - Audio Signal Processing
    1. Concepts
        - Samples
        - Sample rate
        - Duration
        - Channels
    2. Waveforms
    3. FFTs
    4. STFT
    5. Spectrograms
    6. Analyzing stream of audio

In [ ]:
recording_filename = 'recording.wav'

In [ ]:
# Recording audio with sounddevice and soundfile
# 
# https://python-soundfile.readthedocs.io/
# https://python-sounddevice.readthedocs.io

import sounddevice as sd
import soundfile as sf
import numpy as np
import time

# config
samplerate = 44100
duration = 5
channels = 1

print(f"Recording for {duration} seconds at {samplerate} Hz...")

# record audio from the microphone into a numpy array with sounddevice
recording = sd.rec(
    int(duration * samplerate),
    samplerate=samplerate,
    channels=channels,
    dtype='float32'
)
sd.wait()

print(f"Recording finished. Saving to {recording_filename}...")

# save the file with soundfile
sf.write(
    recording_filename,
    recording,
    samplerate,
    subtype='PCM_16'
    )

print(f"File '{recording_filename}' saved successfully.")

In [ ]:
# Playing audio with sounddevice and soundfile
import sounddevice as sd
import soundfile as sf

# data is a numpy array!
data, samplerate = sf.read(
    recording_filename,
    dtype='float32'    
)

print(f"Playing '{recording_filename}'...")
sd.play(data, samplerate)

sd.wait()

print(f"Finished playing '{recording_filename}'.")

In [ ]:
# Display the recording with iPython Display Audio
from IPython.display import Audio, display
Audio(recording_filename) # or Audio(data, rate=sample_rate) from the cell above

In [ ]:
# Look at samples in a recording with NumPy and matplotlib
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf

# load in our recording
data, sample_rate = sf.read(recording_filename)

# get some basic info
print(f"Sample rate: {sample_rate} Hz")
print(f"Duration: : {len(data) / sample_rate:.2f} seconds")
print(f"Number of samples: {len(data)}")
print(f"Shape: {data.shape}") # (n_samples,) for mono (n samples, n_channels) for stereo

# create time axis in seconds
duration = len(data) / sample_rate
time = np.linspace(0, duration, len(data)) # not sure what linspace is? let's look at the documentation!

# plot the waveform
plt.figure(figsize=(12,4))
plt.plot(time, data) # x, y axis
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.title('Audio WaveForm')
plt.grid(True)
plt.tight_layout()
plt.show()




In [ ]:
# Looking at / accessing individual samples

# First 10 samples
print("First 10 samples: ", data[:10])

# Sample at 1 second
sample_index = int(1.0 * sample_rate)
print("Sample at 1 second: ", data[sample_index])

# min and max amplitude
print("Max amplitude: ", np.max(data))
print("Min amplitude: ", np.min(data))

In [ ]:
# Exercise: Silence seconds 2-3 in the recording and play it back


In [ ]:
# Exercise: Reverse the recording and play it back

In [ ]:
# Analyzing sound with FFT

# Origins of FFT
# Discrete Forier transform: https://en.wikipedia.org/wiki/Discrete_Fourier_transform
# Gauss: https://en.wikipedia.org/wiki/Carl_Friedrich_Gauss
# Fourier: https://en.wikipedia.org/wiki/Joseph_Fourier

# Cooley-Tukey FFT Algorithm 
# Original paper: https://www.ams.org/journals/mcom/1965-19-090/S0025-5718-1965-0178586-1/S0025-5718-1965-0178586-1.pdf
# Wikipedia: https://en.wikipedia.org/wiki/Cooley%E2%80%93Tukey_FFT_algorithm

import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf

# load in audio
data, sample_rate = sf.read(recording_filename)

# compute FFT using numpy
# FFT converts time-domain signal to frequency-domain
fft = np.fft.fft(data)

# get magnitude spectrum (absolute values)
# this tells us how much energy is at each frequency
magnitude = np.abs(fft)

# create frequency axis
# np.fft.fftfreq gives us the frequency bins for the FFT output
freq_bins = np.fft.fftfreq(len(data), 1/sample_rate)

# only plot positive frequencies (FFT is symmetric)
# Nyquist theorem: we can only represent frequencies up to sample_rate / 2
positive_freq_idx = freq_bins >= 0
positive_freq = freq_bins[positive_freq_idx]
positive_magnitude = magnitude[positive_freq_idx]

# plot FFT spectrum
plt.figure(figsize=(12, 6))
plt.plot(positive_freq, positive_magnitude)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('FFT Frequency Spectrum (Full Signal)')
plt.grid(True)
plt.xlim(0, 5000) # Focus on 0-5kHz - most audio content is here - comment this out / play with the values!
plt.tight_layout()
plt.show()

print(f"FFT computed on {len(data)} samples")
print(f"Frequency resolution: {sample_rate / len(data):.2f} Hz per bin")
print(f"Max frequency representable: {sample_rate / 2} Hz (Nyquist theorem)") # how would we affect this?




In [ ]:
# Looking at Short-Time Fourier Transform (STFT) with librosa

import librosa
import numpy as np
import matplotlib.pyplot as plt

# load in recording with librosa
y, sr = librosa.load(recording_filename, sr=None)

# compute STFT
# STFT breaks down the signal into overlapping windows and computes FFT
# This gives us frequency content over time (unlike FFT which is frequency for the whole signal)

stft = librosa.stft(y)

# stft is a complex-valued matrix
# Shape: (n_frequencies, n_time_frames)
print(f"STFT shape: {stft.shape}")
print(f"Number of frequency bins: {stft.shape[0]}")
print(f"Number of time frames: {stft.shape[1]}")

# get magnitude (absolute value) of STFT
magnitude = np.abs(stft)

# convert to decibles for better visualization
# this is the poewr spectral density in dB
S_db = librosa.power_to_db(magnitude**2, ref=np.max)

# plot STFT magnitude over time
# we can visualize it as a 2D plot: frequency vs time
plt.figure(figsize=(12,6))
plt.imshow(S_db, aspect='auto', origin='lower', cmap='viridis')
plt.xlabel('Time Frame')
plt.ylabel('Frequency Bin')
plt.title('STFT Magnitude (dB)')
plt.colorbar(label='dB')
plt.tight_layout()
plt.show()

# get actual frequency and time values
frequencies = librosa.fft_frequencies(sr=sr)
times = librosa.frames_to_time(np.arange(S_db.shape[1]), sr=sr )

print(f"Frequency range: {frequencies[0]:.1f} Hz to {frequencies[-1]:.1f} Hz")
print(f"Time range: {times[0]:.2f} s to {times[-1]:.2f} s")

In [ ]:
# Looking at spectrogram

import librosa
import librosa.display
import matplotlib.pyplot as plt

# load audio
y, sr = librosa.load(recording_filename, sr=None)

# compute STFT
D = librosa.stft(y)

# convert to power spectrogram (magnitude squared)
S = np.abs(D)**2

# Convert to decibels
S_db = librosa.power_to_db(S, ref=np.max)

# Visualize spectrogram using librosa's built-in display function
# This automatically handles frequency and time axes correctly
plt.figure(figsize=(12,6))
librosa.display.specshow(
    S_db,
    x_axis='time',
    y_axis='hz',
    sr=sr
    )
plt.colorbar(
    format='%+2.0f db', 
    label='Power (db)'
    )
plt.title('Spectrogram')
plt.tight_layout()
plt.show()

# above is a linar frequency scale
# we can use Mel scale which is more aligneed with our human ear perception of frequency

S_mel = librosa.feature.melspectrogram(y=y, sr=sr)
S_mel_db = librosa.power_to_db(S_mel, ref=np.max)

plt.figure(figsize=(12,6))
librosa.display.specshow(
    S_mel_db,
    x_axis='time',
    y_axis='mel',
    sr=sr
    )
plt.colorbar(
    format='%+2.0f db', 
    label='Power (db)'
    )
plt.title('Mel Spectrogram')
plt.tight_layout()
plt.show()

In [ ]:
# Real time analysis from microphone
# Using sounddevice's callback to process audio as it comes in

import sounddevice as sd
import numpy as np

# config
SAMPLE_RATE = 44100
BLOCK_SIZE = 2048 # number of samples per block

def audio_callback(indata, frames, time, status):
    """This function is called for each audio block"""
    if status:
        print(f"Status: {status}")
    
    # indata is a numpy array of audio samples
    # calculate the volume using Root Mean Square (RMS)
    volume = np.sqrt(np.mean(indata**2))

    # simple volume metter with astrisks
    meter = '*' * int(volume * 200)
    print(f"Volume:  {meter}")

print("Listening to the microphone...stop this cell to stop")

try:
    with sd.InputStream(
        samplerate=SAMPLE_RATE,
        blocksize=BLOCK_SIZE,
        channels=1,
        callback=audio_callback
    ):
        sd.sleep(10000) # listen for 10 seconds

except KeyboardInterrupt:
    print("\nStopped")

In [ ]:
# Fundamental pitch detection using FFT
# Find the strongest frequency in an audio recording

import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt

# load audio data
data, sample_rate = sf.read(recording_filename)

# if stereo, convert to mono
if len(data.shape) > 1:
    data = data[:, 0]

# take a short segment (e.g. 0.1 seconds) for analysis
segment_duration = 0.1 # in seconds
segment_samples = int(segment_duration * sample_rate)
segment = data[:segment_samples]

# compute FFT
fft = np.fft.fft(segment)
magnitude = np.abs(fft)

# get frequency bins
freq_bins = np.fft.fftfreq(len(segment), 1/sample_rate)

# only look at positive frequencies
positive_idx = freq_bins > 0
positive_freq = freq_bins[positive_idx]
positive_magnitude = magnitude[positive_idx]

# find the fundamental frequency (strongest frequency)
peak_idx = np.argmax(positive_magnitude)
fundamental_freq = positive_freq[peak_idx]

print(f"Fundamental frequency: {fundamental_freq:.1f} Hz")

# Plot
plt.figure(figsize=(10,4))
plt.plot(positive_freq, positive_magnitude)
plt.axvline(x=fundamental_freq, color='r', linestyle='--', label=f'Fundamental: {fundamental_freq:.1f} Hz')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Frequency Spectrum with Fundamental Pitch')
plt.xlim(0, 2000) # Focused within speech/music range
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Pitch detection from speech using librosa
# Track how pitch changes over time in a speech recording

# load audio
y, sr = librosa.load(recording_filename)

# use librosa's pyin for pitch detection
# pyin is good for speech and monophonic audio
# fmin and f max set the range for frequencies to look for

f0, voiced_flag, voiced_probs = librosa.pyin(
    y,
    fmin = 50, # min frequency (Hz) - low male voice
    fmax = 500, # maximum frequency (Hz) - high female voice
    sr = sr
)

# f0 contains the detected pitch for each time frame
# NaN (not a number) means no pitch deteted (silence or noise)

# get time values for each frame
times = librosa.times_like(f0, sr=sr)

# plot the pitch over time
plt.figure(figsize=(12, 4))
plt.plot(times, f0, label='Detected Pitch')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.title('Pitch (F0) OVer Time in Speech')
plt.grid(True)
plt.legend()
plt.show()

# print some states
valid_pitches = f0[~np.isnan(f0)] # remove NaN values
if len(valid_pitches) > 0:
    print(f"Average pitch: {np.mean(valid_pitches):.1f} Hz")
    print(f"Pitch range: {np.min(valid_pitches):.1f} - {np.max(valid_pitches):.1f} Hz")
else:
    print("No pitch detected in recording")


In [ ]:
# Detects a specific pitch from microphone and trigger an action
# Prints a message when we detect a pitch near 500hz

import sounddevice as sd
import numpy as np

# config
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
TARGET_FREQ = 500 # frequency to detect
TOLERANCE = 50
MIN_VOLUME = 0.01

def find_fundamental(audio_data, sample_rate):
    """Find the fundamental frequency in audio data using FFT"""

    # Compute FFT
    fft = np.fft.fft(audio_data)
    magnitude = np.abs(fft)

    # get frequency bins
    freq_bins = np.fft.fftfreq(len(audio_data), 1/sample_rate)

    # only look at the positive frequencies betweeen 50-2000 Hz
    valid_idx = (freq_bins > 50) & (freq_bins < 2000)

    if not np.any(valid_idx):
        return 0

    valid_freq = freq_bins[valid_idx]
    valid_magnitude = magnitude[valid_idx]

    # find peak frequency
    peak_idx = np.argmax(valid_magnitude)
    return valid_freq[peak_idx]


def audio_callback(indata, frames, time, status):
    """Called for each audio block"""
    if status:
        print(f'Status: {status}')

    # flatten to 1D array
    audio = indata[:, 0]

    # check if there is enough volume
    volume = np.sqrt(np.mean(audio**2))
    if volume < MIN_VOLUME:
        return

    # find the fundamental frequency
    freq = find_fundamental(audio, SAMPLE_RATE)

    # check if its close to our target
    if abs(freq - TARGET_FREQ) < TOLERANCE:
        print(f"Detected {freq:.0f} Hz! (Target: {TARGET_FREQ} Hz)")

# Start listening
print(f'Listening for ~{TARGET_FREQ} Hz...press Ctrl+C to stop')
print(f'Try whistling or playing a tone :)')

try:
    with sd.InputStream(
        samplerate=SAMPLE_RATE,
        blocksize=BLOCK_SIZE,
        channels=1,
        callback=audio_callback
    ):
        sd.sleep(30000) # listen for 30 seconds
except KeyboardInterrupt:
    print("\nStopped")

# Demo - Radio Rex example with Arduino

In [ ]:
# Send signal to Arduino when pitch is detected

import sounddevice as sd
import numpy as np
import serial
import time

# config
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
TARGET_FREQ = 500 # frequency to detect - refer to Radio Rex example!
TOLERANCE = 50
MIN_VOLUME = 0.01

SERIAL_PORT = '/dev/tty.usbmodem1101' # <--- Change this
BAUD_RATE = 9600



# connect to arduino
try:
    arduino = serial.Serial(SERIAL_PORT, BAUD_RATE)
    time.sleep(2) # wait for arduino to reset
    print(f"Connected to Arduino on {SERIAL_PORT}")
except:
    arduino = None
    print("Could not connect to Arduino - running in test mode")

def find_fundamental(audio_data, sample_rate):
    """Find the fundamental frequency in audio data using FFT"""

    # Compute FFT
    fft = np.fft.fft(audio_data)
    magnitude = np.abs(fft)

    # get frequency bins
    freq_bins = np.fft.fftfreq(len(audio_data), 1/sample_rate)

    # only look at the positive frequencies betweeen 50-2000 Hz
    valid_idx = (freq_bins > 50) & (freq_bins < 2000)

    if not np.any(valid_idx):
        return 0

    valid_freq = freq_bins[valid_idx]
    valid_magnitude = magnitude[valid_idx]

    # find peak frequency
    peak_idx = np.argmax(valid_magnitude)
    return valid_freq[peak_idx]


def audio_callback(indata, frames, time, status):
    """Called for each audio block"""
    if status:
        print(f'Status: {status}')

    # flatten to 1D array
    audio = indata[:, 0]

    # check if there is enough volume
    volume = np.sqrt(np.mean(audio**2))
    if volume < MIN_VOLUME:
        return

    # find the fundamental frequency
    freq = find_fundamental(audio, SAMPLE_RATE)

    # check if its close to our target
    if abs(freq - TARGET_FREQ) < TOLERANCE:
        print(f"Detected {freq:.0f} Hz! (Target: {TARGET_FREQ} Hz)") 
        # SEND TO ARDUINO TO DO SOMETHING COOL!
        if arduino:
            print(f"sending 1 to Arduino")
            arduino.write(b'1') # send '1' to Arduino

    # Start listening
print(f'Listening for ~{TARGET_FREQ} Hz...press Ctrl+C to stop')
print(f'Try whistling or playing a tone :)')
print(f'When detected, Arduino LED will blink!')

try:
    with sd.InputStream(
        samplerate=SAMPLE_RATE,
        blocksize=BLOCK_SIZE,
        channels=1,
        callback=audio_callback
    ):
        sd.sleep(30000) # listen for 30 seconds
except KeyboardInterrupt:
    print("\nStopped")
finally:
    if arduino:
        arduino.close()
# 

In [ ]:
# Arduino code - copy + paste and load onto Arduino

f"""

const int LED_PIN = 13; // built-in LED on most Arduinos

void setup() {
    Serial.begin(9600);
    pinMode(LED_PIN, OUTPUT);
    digitalWrite(LED_PIN, LOW);
}

void loop() {
    // check if data received from Python
    if (Serial.available() > 0) {
        char received = Seria.read();

        if (received == '1') {
            // blink the LED
            digitalWrite(LED_PIN, HIGH);
            delay(200);
            digitalWRite(LED_PIN, LOW);
        }
    }
}

"""

In [ ]:
# Exercise: Make something activate with pitch (simple)
# Consider sending signal to a p5 sketch over OSC for example

In [ ]:
# Exercise: Make something activate with speech and have it activate something physical in the world (harder!) 
# Hint: Analyze a speech recording and determine a fundamental frequency / format in that speech utterance
# For example, with Radio Rex, it was "concluded that Rex was sensitive to vowels with resonances in the 500 Hz range"

In [ ]:
# Extra Credit

# Playing with playback speed, volume, other effects
# Reconstructing sound with iFFT
# Look at auto correleation
# Pitch detection
# Other kinds of time/frequency/amplitude functions